In [31]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
df = pd.read_excel('poblacion_localidades.xlsx', skiprows=11)
num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = df.select_dtypes(include='object').columns.tolist()
df.head()

,COD_LOC,NOM_LOC,AREA,AÑO,Hombres_0,Hombres_1,Hombres_2,Hombres_3,Hombres_4,Hombres_5,...,Total_94,Total_95,Total_96,Total_97,Total_98,Total_99,Total_100 y más,TOTAL HOMBRES,TOTAL MUJERES,TOTAL
0,1,Usaquén,Cabecera Municipal,2018,3043,2970,2958,2931,2790,2799,...,330,290,192,182,146,144,550,246729,288476,535205
1,1,Usaquén,Centros Poblados y Rural Disperso,2018,0,5,4,2,4,2,...,0,0,0,0,0,2,0,208,280,488
2,1,Usaquén,Total,2018,3043,2975,2962,2933,2794,2801,...,330,290,192,182,146,146,550,246937,288756,535693
3,2,Chapinero,Cabecera Municipal,2018,937,911,814,725,673,684,...,123,84,75,64,51,54,270,73954,81665,155619
4,2,Chapinero,Centros Poblados y Rural Disperso,2018,7,12,2,12,7,10,...,0,0,0,0,0,0,0,448,412,860


Para estos datos poblacionales del DANE unicamente nos interesan las columnas: COD_LOC, NOM_LOC, AÑO, TOTAL
Y unicamente nos interesan las filas que tengan AREA = Total ya que eso nos da la polbacion total de ese año sobre la localidad.

In [33]:
## Seleccion de Columnas y Filas de interes
df = df[['COD_LOC', 'NOM_LOC', 'AÑO', 'AREA', 'TOTAL']]
df = df[df['AREA'] == 'Total']
df = df.drop(columns=['AREA'])
df.head()

,COD_LOC,NOM_LOC,AÑO,TOTAL
2,1,Usaquén,2018,535693
5,2,Chapinero,2018,156479
8,3,Santa Fe,2018,103985
11,4,San Cristóbal,2018,385514
14,5,Usme,2018,363394


Ahora vamos a cambiar los nombres de las columnas para mantener un formato constante. Todo en mayusculas y si hay tildes se pone la letra sin tilde y si hay ñ por ejemplo AÑO seria ANIO. 

In [34]:
nuevas_columnas = {
    'COD_LOC': 'ID_LOCALIDAD',
    'NOM_LOC': 'LOCALIDAD',
    'AÑO': 'ANIO',
    'TOTAL': 'TOTAL'
}
df = df.rename(columns=nuevas_columnas)
df.head()

,ID_LOCALIDAD,LOCALIDAD,ANIO,TOTAL
2,1,Usaquén,2018,535693
5,2,Chapinero,2018,156479
8,3,Santa Fe,2018,103985
11,4,San Cristóbal,2018,385514
14,5,Usme,2018,363394


Ahora lo que vamos a hacer es agrupar que cada fila contenga lo siguiente, el ID_LOCALIDAD, LOCALIDAD, y POBLACION_2020, POBLACION_2021, POBLACION_2022, POBLACION_2023 Y POBLACION_2024. Donde en cada POBLACION_202X tenga el valor de la poblacion total de ese año.

In [35]:
# Ajuste final para tener una fila por localidad con las poblaciones de cada año en columnas separadas
df_pivot = df.pivot(index=['ID_LOCALIDAD', 'LOCALIDAD'], columns='ANIO', values='TOTAL').reset_index()
df_pivot.columns = ['ID_LOCALIDAD', 'LOCALIDAD'] + [f'POBLACION_{col}' for col in df_pivot.columns[2:]]
df_pivot.head()

,ID_LOCALIDAD,LOCALIDAD,POBLACION_2018,POBLACION_2019,POBLACION_2020,POBLACION_2021,POBLACION_2022,POBLACION_2023,POBLACION_2024,POBLACION_2025,POBLACION_2026,POBLACION_2027,POBLACION_2028,POBLACION_2029,POBLACION_2030,POBLACION_2031,POBLACION_2032,POBLACION_2033,POBLACION_2034,POBLACION_2035
0,1,Usaquén,535693,550706,564539,571268,579447,586954,594611,602412,611076,619833,628528,637349,646032,654728,663307,671667,679896,687885
1,2,Chapinero,156479,163148,169786,173353,176471,179406,182103,184757,187205,189552,191679,193776,195687,197595,199076,200876,202642,204121
2,3,Santa Fe,103985,105926,107458,107784,107630,107677,107906,107851,108476,108810,109157,109687,110457,111466,112454,113584,114702,115972
3,4,San Cristóbal,385514,392322,397410,401060,403674,406498,409106,411570,413822,416085,417870,419888,423132,426335,429676,432944,436087,439049
4,5,Usme,363394,374887,384943,393366,400580,407645,414995,422489,429958,437488,444960,452517,462288,470469,478567,486565,494507,502274


In [36]:
selected_years = [2020, 2021, 2022, 2023, 2024]
df_final = df_pivot[['ID_LOCALIDAD', 'LOCALIDAD'] + [f'POBLACION_{year}' for year in selected_years]]
df_final.head()

,ID_LOCALIDAD,LOCALIDAD,POBLACION_2020,POBLACION_2021,POBLACION_2022,POBLACION_2023,POBLACION_2024
0,1,Usaquén,564539,571268,579447,586954,594611
1,2,Chapinero,169786,173353,176471,179406,182103
2,3,Santa Fe,107458,107784,107630,107677,107906
3,4,San Cristóbal,397410,401060,403674,406498,409106
4,5,Usme,384943,393366,400580,407645,414995


In [39]:
# vamos a cambiar los valores de los nombres de las localidades para que no tengan acentos ni ñ
import unidecode

df_final["LOCALIDAD"] = df_final["LOCALIDAD"].apply(unidecode.unidecode)
df_final.head()

C:\Users\yarso\AppData\Local\Temp\ipykernel_29640\611153413.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["LOCALIDAD"] = df_final["LOCALIDAD"].apply(unidecode.unidecode)


,ID_LOCALIDAD,LOCALIDAD,POBLACION_2020,POBLACION_2021,POBLACION_2022,POBLACION_2023,POBLACION_2024
0,1,Usaquen,564539,571268,579447,586954,594611
1,2,Chapinero,169786,173353,176471,179406,182103
2,3,Santa Fe,107458,107784,107630,107677,107906
3,4,San Cristobal,397410,401060,403674,406498,409106
4,5,Usme,384943,393366,400580,407645,414995


In [40]:
# Exportar el DataFrame final a un nuevo archivo csv
df_final.to_csv('poblacion_localidades_consolidada.csv', index=False)

In [41]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID_LOCALIDAD    20 non-null     int64 
 1   LOCALIDAD       20 non-null     object
 2   POBLACION_2020  20 non-null     int64 
 3   POBLACION_2021  20 non-null     int64 
 4   POBLACION_2022  20 non-null     int64 
 5   POBLACION_2023  20 non-null     int64 
 6   POBLACION_2024  20 non-null     int64 
dtypes: int64(6), object(1)
memory usage: 1.2+ KB
